# Support Vector Machine and Random Trees (Random Forest) Classification

This notebook is to be used with the ArcGIS Pro Python API to classify SENTINEL-2 images into [CORINE](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018) land cover classes using SVM and RF classifiers.

Author: Adian Dawuda | adian.dawuda@stud.plus.ac.at

In [8]:
# Import system modules
import arcpy
from arcpy.ia import *

In [2]:
# Clip Sentinel 2 raster to San Marino Boundaries
arcpy.management.Clip(
    "SanMarinoS2_full.tif", 
    "-20037507.0671618 -30240971.9583861 20037507.0671618 18460513.2470149", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_all_bands", 
    "World_Countries", 
    "1.79e+308", 
    "ClippingGeometry", 
    "NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\Adian\\Documents\\ArcGIS\\Projects\\AnalysisAndModelingFinal\\AnalysisAndModelingFinal.gdb\\SanMarinoS2_all_bands'>

In [1]:
# Create a copy of the image with bands 2, 3, 4, and 8
arcpy.management.MakeRasterLayer(
    "SanMarinoS2_all_bands", 
    "SanMarinoS2", 
    '', 
    '1378670 5448230 1394740 5465090 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', 
    "2;3;4;8"
)

<Result 'SanMarinoS2'>

## SVM

In [ ]:
# Train SVM and Generate an Esri classifier definition file (.ecd)
arcpy.ia.TrainSupportVectorMachineClassifier(
    "SanMarinoS2", 
    "samples", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\SanMarinoTrainedSVM.ecd", 
    '', 
    500, 
    "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY", 
    None
)

In [6]:
# Classify the Sentinel image using the Esri classifier definition file (.ecd)
out_raster_dataset = arcpy.ia.ClassifyRaster(
    "SanMarinoS2.tif",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\SanMarinoTrainedSVM.ecd",
    ''
);
out_raster_dataset.save(
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\LandCoverSanMarinoSVM.crf"
)

## RF

In [18]:
# Train RF and Classify raster
arcpy.stats.Forest(
    "PREDICT_RASTER", 
    "samples", 
    "Classcode", 
    "CATEGORICAL", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_4 #;C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_3 #;C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_2 #;C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_8 #", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\rf10.tif", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_4 C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_4;C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_3 C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_3;C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_2 C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_2;C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_8 C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2\Band_8", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\TrainedFeatures10.shp", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\VariableImportance10.dbf", 
    "TRUE", 
    100, 
    None, 
    None, 
    100, 
    None, 
    10, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\classificationPerformanceTable10.dbf", 
    None, 
    "TRUE", 
    1, 
    "FALSE"
)

<geoprocessing server result object object at 0x000001F0B0950030>

## Accuracy Assessment

In [20]:
# Convert CLC to raster format
arcpy.conversion.PolygonToRaster(
    "CORINE_LC_San_Marino", 
    "Code_18", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\CORINE_Raster", 
    "CELL_CENTER", 
    "NONE", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2", 
    "DO_NOT_BUILD"
)

<Result 'C:\\Users\\Adian\\Documents\\ArcGIS\\Projects\\AnalysisAndModelingFinal\\AnalysisAndModelingFinal.gdb\\CORINE_Raster'>

### SVM

In [26]:
# Create points for SVM
arcpy.ia.CreateAccuracyAssessmentPoints(
    "LandCoverSanMarinoSVM", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsSVMPrelim.shp", 
    "CLASSIFIED", 
    500, 
    "STRATIFIED_RANDOM",
    None
)

<geoprocessing server result object object at 0x000001F05367DB10>

In [27]:
# Update points with ground truth identifiers
arcpy.ia.UpdateAccuracyAssessmentPoints(
    "CORINE_Raster", 
    "AccuracyAssessmentPointsSVMPrelim", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsSVM.shp", 
    "GROUND_TRUTH", 
    None, 
    None
)

<geoprocessing server result object object at 0x000001F05367DDB0>

In [28]:
# Update ground truth identifiers with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal"

feature_class = "AccuracyAssessmentPointsSVM"

# Field name to be updated
field_name = "GrndTruth"


# Map values to match CLC
value_mapping = {
    1: 211,
    2: 112,
    3: 324,
    4: 243,
    5: 242,
    6: 311,
    7: 313,
    8: 231,
    9: 321,
    10: 121,
    11: 142
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [29]:
# Calculate confusion Matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsSVM", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\AccuracySVM"
)

<geoprocessing server result object object at 0x000001F0A0D4B810>

### RF

In [3]:
# Create points for RF
# The classification layer must have 8 or 16 bpp
arcpy.ia.CreateAccuracyAssessmentPoints(
    "rf10_16bit.crf", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsRFPrelim.shp", 
    "CLASSIFIED", 
    500, 
    "STRATIFIED_RANDOM",
    None
)

<geoprocessing server result object object at 0x00000257D1C47AE0>

In [4]:
# Update classification with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal"

feature_class = "AccuracyAssessmentPointsRFPrelim"

# Field name to be updated
field_name = "Classified"


# Map values to match CLC
value_mapping = {
    0: 112,
    1: 121,
    2: 142,
    3: 211,
    4: 231,
    5: 242,
    6: 243,
    7: 311,
    8: 313,
    9: 321,
    10: 324
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [5]:
# Update points with ground truth identifiers
arcpy.ia.UpdateAccuracyAssessmentPoints(
    "CORINE_Raster", 
    "AccuracyAssessmentPointsRFPrelim", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsRF.shp", 
    "GROUND_TRUTH", 
    None, 
    None
)

<geoprocessing server result object object at 0x00000257CD337A50>

In [6]:
# Update ground truth identifiers with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal"

feature_class = "AccuracyAssessmentPointsRF"

# Field name to be updated
field_name = "GrndTruth"


# Map values to match CLC
value_mapping = {
    1: 211,
    2: 112,
    3: 324,
    4: 243,
    5: 242,
    6: 311,
    7: 313,
    8: 231,
    9: 321,
    10: 121,
    11: 142
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [7]:
# Calculate confusion Matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsRF", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\AccuracyRF"
)

<geoprocessing server result object object at 0x000002581FFF9C30>

# 60m resolution analysis

In [ ]:
# Resample image
arcpy.management.Resample(
    "SanMarinoS2_all_bands", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m", 
    "60 60", 
    "NEAREST"
)

## SVM

In [ ]:
# Train SVM and Generate an Esri classifier definition file (.ecd)
arcpy.ia.TrainSupportVectorMachineClassifier(
    "SanMarinoS2_60m", 
    "samples", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\SanMarinoTrainedSVM_60m.ecd", 
    '', 
    500, 
    "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY", 
    None
)

In [41]:
# Classify the Sentinel image using the Esri classifier definition file (.ecd)
out_raster_dataset = arcpy.ia.ClassifyRaster(
    "SanMarinoS2_60m",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\SanMarinoTrainedSVM_60m.ecd",
    ''
);
out_raster_dataset.save(
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\LandCoverSanMarinoSVM_60m.crf"
)

## RF

In [42]:
# Train RF and Classify raster
arcpy.stats.Forest(
    "PREDICT_RASTER", 
    "samples", 
    "Classcode", 
    "CATEGORICAL", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m #", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\rf60.crf", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m", r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\TrainedFeatures60.shp", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\VariableImportance60.dbf", 
    "TRUE", 
    100, 
    None, 
    None, 
    100, 
    None, 
    10, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\classificationPerformanceTable60.dbf", 
    None, 
    "FALSE", 
    1, 
    "FALSE"
)

<geoprocessing server result object object at 0x000001F0A0D4B990>

## Accuracy Assessment

### SVM

In [45]:
# Create points for SVM
arcpy.ia.CreateAccuracyAssessmentPoints(
    "LandCoverSanMarinoSVM_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsSVMPrelim_60m.shp", 
    "CLASSIFIED", 
    500, 
    "STRATIFIED_RANDOM",
    None
)

<geoprocessing server result object object at 0x000001F0A0D4BAB0>

In [46]:
# Update points with ground truth identifiers
arcpy.ia.UpdateAccuracyAssessmentPoints(
    "CORINE_Raster", 
    "AccuracyAssessmentPointsSVMPrelim_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsSVM_60m.shp", 
    "GROUND_TRUTH", 
    None, 
    None
)

<geoprocessing server result object object at 0x000001F0A0D4BA50>

In [47]:
# Update ground truth identifiers with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal"

feature_class = "AccuracyAssessmentPointsSVM_60m"

# Field name to be updated
field_name = "GrndTruth"


# Map values to match CLC
value_mapping = {
    1: 211,
    2: 112,
    3: 324,
    4: 243,
    5: 242,
    6: 311,
    7: 313,
    8: 231,
    9: 321,
    10: 121,
    11: 142
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [48]:
# Calculate confusion Matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsSVM_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\AccuracySVM_60m"
)

<geoprocessing server result object object at 0x000001F0A002F4B0>

### RF

In [1]:
# Create points for RF
# RF classification layer must be 8 or 16 bpp
arcpy.ia.CreateAccuracyAssessmentPoints(
    "rf60_16bit.crf", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsRFPrelim_60m.shp", 
    "CLASSIFIED", 
    500, 
    "STRATIFIED_RANDOM", 
    None
)

<geoprocessing server result object object at 0x000002BC3E945840>

In [3]:
# Update classification with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal"

feature_class = "AccuracyAssessmentPointsRFPrelim_60m.shp"

# Field name to be updated
field_name = "Classified"


# Map values to match CLC
value_mapping = {
    0: 112,
    1: 121,
    2: 142,
    3: 211,
    4: 231,
    5: 242,
    6: 243,
    7: 311,
    8: 313,
    9: 321,
    10: 324
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [4]:
# Update points with ground truth identifiers
arcpy.ia.UpdateAccuracyAssessmentPoints(
    "CORINE_Raster", 
    "AccuracyAssessmentPointsRFPrelim_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AccuracyAssessmentPointsRF_60m.shp", 
    "GROUND_TRUTH", 
    None, 
    None
)

<geoprocessing server result object object at 0x000002BC391AFC30>

In [5]:
# Update ground truth identifiers with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal"

feature_class = "AccuracyAssessmentPointsRF_60m"

# Field name to be updated
field_name = "GrndTruth"


# Map values to match CLC
value_mapping = {
    1: 211,
    2: 112,
    3: 324,
    4: 243,
    5: 242,
    6: 311,
    7: 313,
    8: 231,
    9: 321,
    10: 121,
    11: 142
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [6]:
# Calculate confusion Matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsRF_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\AccuracyRF_60m"
)

<geoprocessing server result object object at 0x000002BC3EAF2CF0>